# Unitree Mujoco demo



## Open virtual display

In [ ]:
from utils import display_desktop
display_desktop()

## Set parameters

In [ ]:
import config

config.ROBOT = "go2" # Robot name, "go2", "b2", "b2w", "h1", "go2w", "g1" 
config.ROBOT_SCENE = "../unitree_robots/" + config.ROBOT + "/scene_terrain.xml" # Robot scene
config.USE_JOYSTICK = 0 # Simulate Unitree WirelessController using a gamepad

## Start Mujoco Simulation

In [ ]:
from threading import Thread

from unitree_mujoco import PhysicsViewerThread, SimulationThread

viewer_thread = Thread(target=PhysicsViewerThread)
sim_thread = Thread(target=SimulationThread)

viewer_thread.start()
sim_thread.start()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


1741881585.297482 [1]     python: selected interface "lo" is not multicast-capable: disabling multicast


 
<<------------- Link ------------->> 
link_index: 0 , name: world
link_index: 1 , name: base_link
link_index: 2 , name: FL_hip
link_index: 3 , name: FL_thigh
link_index: 4 , name: FL_calf
link_index: 5 , name: FL_foot
link_index: 6 , name: FR_hip
link_index: 7 , name: FR_thigh
link_index: 8 , name: FR_calf
link_index: 9 , name: FR_foot
link_index: 10 , name: RL_hip
link_index: 11 , name: RL_thigh
link_index: 12 , name: RL_calf
link_index: 13 , name: RL_foot
link_index: 14 , name: RR_hip
link_index: 15 , name: RR_thigh
link_index: 16 , name: RR_calf
link_index: 17 , name: RR_foot
 
<<------------- Joint ------------->> 
joint_index: 1 , name: FL_hip_joint
joint_index: 2 , name: FL_thigh_joint
joint_index: 3 , name: FL_calf_joint
joint_index: 4 , name: FR_hip_joint
joint_index: 5 , name: FR_thigh_joint
joint_index: 6 , name: FR_calf_joint
joint_index: 7 , name: RL_hip_joint
joint_index: 8 , name: RL_thigh_joint
joint_index: 9 , name: RL_calf_joint
joint_index: 10 , name: RR_hip_joint
j

## Go2 Stand up and down

In [ ]:
import time
import sys
import numpy as np

from unitree_sdk2py.core.channel import ChannelPublisher, ChannelSubscriber
from unitree_sdk2py.core.channel import ChannelFactoryInitialize
from unitree_sdk2py.idl.default import unitree_go_msg_dds__LowCmd_
from unitree_sdk2py.idl.unitree_go.msg.dds_ import LowCmd_
from unitree_sdk2py.utils.crc import CRC

stand_up_joint_pos = np.array([
    0.00571868, 0.608813, -1.21763, -0.00571868, 0.608813, -1.21763,
    0.00571868, 0.608813, -1.21763, -0.00571868, 0.608813, -1.21763
],
                              dtype=float)

stand_down_joint_pos = np.array([
    0.0473455, 1.22187, -2.44375, -0.0473455, 1.22187, -2.44375, 0.0473455,
    1.22187, -2.44375, -0.0473455, 1.22187, -2.44375
],
                                dtype=float)

dt = 0.002
runing_time = 0.0
crc = CRC()


# Create a publisher to publish the data defined in UserData class
pub = ChannelPublisher("rt/lowcmd", LowCmd_)
pub.Init()

cmd = unitree_go_msg_dds__LowCmd_()
cmd.head[0] = 0xFE
cmd.head[1] = 0xEF
cmd.level_flag = 0xFF
cmd.gpio = 0
for i in range(20):
    cmd.motor_cmd[i].mode = 0x01  # (PMSM) mode
    cmd.motor_cmd[i].q = 0.0
    cmd.motor_cmd[i].kp = 0.0
    cmd.motor_cmd[i].dq = 0.0
    cmd.motor_cmd[i].kd = 0.0
    cmd.motor_cmd[i].tau = 0.0

while True:
    step_start = time.perf_counter()
    
    runing_time += dt
    # Stop after 10 seconds
    if (runing_time > 10.0):
        break
    if (runing_time < 3.0):
        # Stand up in first 3 second
        
        # Total time for standing up or standing down is about 1.2s
        phase = np.tanh(runing_time / 1.2)
        for i in range(12):
            cmd.motor_cmd[i].q = phase * stand_up_joint_pos[i] + (
                1 - phase) * stand_down_joint_pos[i]
            cmd.motor_cmd[i].kp = phase * 50.0 + (1 - phase) * 20.0
            cmd.motor_cmd[i].dq = 0.0
            cmd.motor_cmd[i].kd = 3.5
            cmd.motor_cmd[i].tau = 0.0
    else:
        # Then stand down
        phase = np.tanh((runing_time - 3.0) / 1.2)
        for i in range(12):
            cmd.motor_cmd[i].q = phase * stand_down_joint_pos[i] + (
                1 - phase) * stand_up_joint_pos[i]
            cmd.motor_cmd[i].kp = 50.0
            cmd.motor_cmd[i].dq = 0.0
            cmd.motor_cmd[i].kd = 3.5
            cmd.motor_cmd[i].tau = 0.0
    
    cmd.crc = crc.Crc(cmd)
    pub.Write(cmd)
    time_until_next_step = dt - (time.perf_counter() - step_start)
    if time_until_next_step > 0:
        time.sleep(time_until_next_step)
